In [1]:
import numpy as np
import pandas as pd
import re
import time
import matplotlib as plt
%matplotlib inline
import seaborn as sns

In [2]:
sample_submission = pd.read_csv('sample_submission.csv')
train_data = pd.read_csv('train_2016_v2.csv')
properties_data= pd.read_csv('properties_2016.csv')
zillow_dict = pd.read_excel('zillow_data_dictionary.xlsx', sheetname=None)

/Users/kaijiang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sample_submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [4]:
train_data.head()
# what if transaction date is also a component for price?
# is it possible to predict transaction date first??

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [34]:
#properties_data.head()
properties_data.decktypeid[pd.isnull(properties_data.decktypeid)==False].unique()
# need to join properties and train

array([ 66.])

In [79]:
len(properties_data)


2985217

In [72]:
properties_data.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [67]:
train_data[pd.isnull(train_data['logerror'])]

,parcelid,logerror,transactiondate


In [54]:
#properties_data.taxdelinquencyyear.value_counts()

In [17]:
zillow_dict.keys()

dict_keys(['BuildingClassTypeID', 'Data Dictionary', 'ArchitecturalStyleTypeID', 'AirConditioningTypeID', 'PropertyLandUseTypeID', 'HeatingOrSystemTypeID', 'TypeConstructionTypeID', 'StoryTypeID'])

In [20]:
zillow_dict['Data Dictionary'].head()

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home


In [21]:
len(train_data)
#len(train_data.parcelid.unique()) # 90150
# there are some, not many, 125 or less houses sold multiple times in train _data

90275

In [27]:
print(len(properties_data.columns))
for i in properties_data.columns:
    print (i , ':\n', len(properties_data[i].unique()))

58
parcelid :
 2985217
airconditioningtypeid :
 8
architecturalstyletypeid :
 9
basementsqft :
 752
bathroomcnt :
 38
bedroomcnt :
 22
buildingclasstypeid :
 6
buildingqualitytypeid :
 13
calculatedbathnbr :
 35
decktypeid :
 2
finishedfloor1squarefeet :
 4605
calculatedfinishedsquarefeet :
 12588
finishedsquarefeet12 :
 10671
finishedsquarefeet13 :
 251
finishedsquarefeet15 :
 8817
finishedsquarefeet50 :
 4803
finishedsquarefeet6 :
 4252
fips :
 4
fireplacecnt :
 10
fullbathcnt :
 21
garagecarcnt :
 25
garagetotalsqft :
 2241
hashottuborspa :
 2
heatingorsystemtypeid :
 15
latitude :
 852326
longitude :
 1042551
lotsizesquarefeet :
 69954
poolcnt :
 2
poolsizesum :
 908
pooltypeid10 :
 2
pooltypeid2 :
 2
pooltypeid7 :
 2
propertycountylandusecode :
 241
propertylandusetypeid :
 16
propertyzoningdesc :
 5639
rawcensustractandblock :
 99394
regionidcity :
 187
regionidcounty :
 4
regionidneighborhood :
 529
regionidzip :
 406
roomcnt :
 37
storytypeid :
 2
threequarterbathnbr :
 8
typec

In [3]:
con_columns = []
cat_columns = []
text_columns = []
id=['parcelid']


for i in range(len(zillow_dict['Data Dictionary'].variable)):
    if zillow_dict['Data Dictionary'].variable[i]=='con':
        con_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))
    if zillow_dict['Data Dictionary'].variable[i]=='cat':
        cat_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))
    if zillow_dict['Data Dictionary'].variable[i]=='text':
        text_columns.append(zillow_dict['Data Dictionary'].Feature[i].replace("'",""))
        
        

In [64]:
text_columns

['fips',
 'latitude',
 'longitude',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'censustractandblock',
 'regionidcounty',
 'regionidcity',
 'regionidzip',
 'regionidneighborhood']

In [4]:
# then read the rule and join to get full data, then use simple random forest to get predicted results, 
# results same for all time points
groundtruth = train_data.merge(properties_data,right_on='parcelid',left_on='parcelid', how='left')
full_data = train_data.merge(properties_data,right_on='parcelid',left_on='parcelid', how='right')


In [119]:
full_data.tail()
# some records are completely missing properties?

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
2985337,168176230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985338,14273630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985339,168040630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985340,168040830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985341,168040430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# easiest baseline model?
# try linear regression with no text features 
# only OHE on categorical and some nomorlization on continous variables

In [5]:
full_data = full_data.replace(np.nan, -1)

In [6]:
full_data['taxdelinquencyflag']=full_data['taxdelinquencyflag'].replace('Y', 1)

In [7]:

#full_data[pd.isnull(full_data)]  = 'null'
from sklearn import preprocessing
for i in cat_columns:
    
    le = preprocessing.LabelEncoder()
    le.fit(full_data[i])
    labels = dict(zip(full_data[i].unique()
               , le.transform(full_data[i].unique())))
    print(i)
    print(labels)
    #print(le.classes_)
    full_data[i]=le.transform(full_data[i])
    

airconditioningtypeid
{1.0: 1, -1.0: 0, 5.0: 3, 13.0: 7, 11.0: 5, 9.0: 4, 3.0: 2, 12.0: 6}
architecturalstyletypeid
{-1.0: 0, 7.0: 4, 8.0: 5, 2.0: 1, 10.0: 6, 3.0: 2, 21.0: 7, 5.0: 3, 27.0: 8}
buildingclasstypeid
{-1.0: 0, 4.0: 4, 3.0: 3, 5.0: 5, 2.0: 2, 1.0: 1}
decktypeid
{-1.0: 0, 66.0: 1}
fireplaceflag
{-1: 0, True: 1}
hashottuborspa
{-1: 0, True: 1}
heatingorsystemtypeid
{2.0: 2, -1.0: 0, 7.0: 4, 6.0: 3, 24.0: 14, 13.0: 8, 20.0: 12, 18.0: 10, 11.0: 6, 1.0: 1, 14.0: 9, 12.0: 7, 10.0: 5, 21.0: 13, 19.0: 11}
pooltypeid10
{-1.0: 0, 1.0: 1}
pooltypeid2
{-1.0: 0, 1.0: 1}
pooltypeid7
{-1.0: 0, 1.0: 1}
propertylandusetypeid
{261.0: 7, 266.0: 11, 265.0: 10, 246.0: 3, 263.0: 8, 269.0: 13, 248.0: 5, 247.0: 4, 267.0: 12, 275.0: 15, 260.0: 6, 31.0: 1, 264.0: 9, 47.0: 2, 270.0: 14, -1.0: 0}
storytypeid
{-1.0: 0, 7.0: 1}
typeconstructiontypeid
{-1.0: 0, 6.0: 2, 13.0: 5, 4.0: 1, 10.0: 3, 11.0: 4}
taxdelinquencyflag
{-1: 0, 1: 1}


In [8]:

OHE = preprocessing.OneHotEncoder()
full_data_ohe=OHE.fit_transform(full_data[cat_columns])

In [9]:
full_data_ohe.shape

(2985342, 76)

In [10]:
# standard scaling
# 这个对skew没有影响
SSL = preprocessing.StandardScaler()
for num_col in con_columns:
    full_data[num_col] = SSL.fit_transform(full_data[num_col])

/Users/kaijiang/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/kaijiang/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/kaijiang/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1)

In [31]:
from scipy.stats import skew, boxcox
skewed_cols = full_data[con_columns].apply(lambda x: skew(x.dropna()))
print (skewed_cols.sort_values())
#这skew程度。。。

assessmentyear                   -16.061851
yearbuilt                         -6.768472
buildingqualitytypeid             -0.179576
bedroomcnt                         0.389654
calculatedbathnbr                  0.403662
fullbathcnt                        0.457064
bathroomcnt                        1.052704
garagecarcnt                       1.157229
numberofstories                    1.552897
roomcnt                            1.598950
poolcnt                            1.725644
garagetotalsqft                    2.115936
threequarterbathnbr                2.617953
fireplacecnt                       2.914606
finishedfloor1squarefeet           5.320149
finishedsquarefeet50               5.699379
taxdelinquencyyear                 8.041079
finishedsquarefeet12              13.855505
yardbuildingsqft17                14.082945
finishedsquarefeet13              21.704447
poolsizesum                       23.503220
landtaxvaluedollarcnt             31.287275
taxvaluedollarcnt               

In [34]:
# do boxcox but input must be positive
for i in con_columns:
    print(min(full_data[i]))

-0.0179600145384
-2.92257093025
-3.13749298291
-1.24129420394
-2.66039639171
-0.34075586133
-0.243858086122
-0.986009312696
-0.0257211580648
-1.51282920365
-0.0485783463575
-0.114349248831
-0.241882422685
-0.333693793969
-2.6472333627
-0.627765209855
-0.516053069986
-0.0643927930118
-0.527248231605
-0.457958841922
-0.0912218175403
-0.868420546853
-0.644036503001
-0.133913333697
-0.0179532740997
-6.96200251788
-0.573397142264
-0.420388723647
-0.558849912707
-0.581600365317
-16.1238723252
-0.136778836581


In [12]:
from scipy.stats import skew, boxcox
for i in con_columns:
    
    full_data[i], _ = boxcox(full_data[i]+20)

/Users/kaijiang/anaconda/lib/python3.6/site-packages/scipy/stats/morestats.py:905: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))


In [36]:
# now we check skewness again
skewed_cols = full_data[con_columns].apply(lambda x: skew(x.dropna()))
print (skewed_cols.sort_values())
# 好很多哎！！！！

yearbuilt                       -1.419290
buildingqualitytypeid           -0.093693
finishedsquarefeet12            -0.078712
bathroomcnt                     -0.068349
unitcnt                         -0.052944
bedroomcnt                      -0.002084
fullbathcnt                      0.000102
calculatedbathnbr                0.007974
calculatedfinishedsquarefeet     0.492046
threequarterbathnbr              0.774685
garagecarcnt                     0.945860
numberofstories                  1.328242
landtaxvaluedollarcnt            1.340946
garagetotalsqft                  1.367154
roomcnt                          1.403018
taxamount                        1.539421
taxvaluedollarcnt                1.610823
poolcnt                          1.692791
structuretaxvaluedollarcnt       1.980638
fireplacecnt                     2.602110
finishedfloor1squarefeet         3.482961
finishedsquarefeet50             3.651832
finishedsquarefeet15             3.996374
yardbuildingsqft17               6

/Users/kaijiang/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:935: RuntimeWarning: overflow encountered in multiply
  s *= a_zero_mean


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

/Users/kaijiang/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
y=full_data['logerror'][:train_data.shape[0]]

In [14]:
#这样时不做ohe只做lable encoding
X= pd.concat([full_data[:train_data.shape[0]][cat_columns], full_data[:train_data.shape[0]][con_columns]], axis=1)

#这样是用ohe的结果
#import scipy
#X= scipy.sparse.hstack([full_data_sparse[:train_data.shape[0]], full_data[:train_data.shape[0]][con_columns]])

# linear models dont accept sparse matrix cuz of nan

In [15]:
X.shape

(90275, 46)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [75]:
from sklearn import linear_model
linear_reg = linear_model.LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [77]:
y_predict = linear_reg.predict(X_test)

In [79]:
metrics.mean_absolute_error(y_test, y_predict)
# 垫底的一个分数。。。果然linear regression 无用哈哈哈
#不过pipeline出来了

0.067832358703616077

In [19]:
from sklearn import grid_search, metrics

#scorer = metrics.make_scorer('ndcg_scorer', needs_proba=True) 
# 如果你要customize scorer

def search_model(train_x, train_y, est, param_grid, n_jobs, cv, refit=False):
##Grid Search for the best model
    model = grid_search.GridSearchCV(estimator  = est,
                                     param_grid = param_grid,
                                     scoring = 'neg_mean_absolute_error',
                                     verbose  = 1,
                                     n_jobs  = n_jobs,
                                     iid    = True,
                                     refit    = refit,
                                     cv      = cv)
    # Fit Grid Search Model
    model.fit(train_x, train_y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:", model.best_params_)
    print("Scores:", model.grid_scores_)
    return model

/Users/kaijiang/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
import xgboost as xgb
param_grid = {
              'learning_rate':[0.02],
              'n_estimators':[500],
              'max_depth': [9],
              'min_child_weight':[50],
              'subsample': [0.78],
              'colsample_bytree':[0.67],
              'gamma':[0.9],
              'nthread': [-1],
              'seed' : [1234]}

model = search_model(X_train,
                     y_train,
                     xgb.XGBRegressor(),
                     param_grid,
                     n_jobs = 1,
                     cv = 4,
                     refit = True)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
